# ☠️ **Módulo 3: Data Poisoning e Modelo Inversion**

> **Tá, mas o que é Data Poisoning?** É como envenenar a comida de alguém para que ela fique doente! Mas em vez de comida, estamos falando dos **dados de treinamento** da IA. Imagine que você tem um funcionário que aprende com exemplos, mas alguém coloca exemplos falsos para ensinar ele a fazer coisas erradas.

---

## **Aula 3.1: O que é Data Poisoning?**

### **Tá, mas como funciona isso?**

Data Poisoning é como ser um **professor malvado** que ensina coisas erradas para um aluno. Em vez de ensinar matemática correta, você ensina que 2+2=5. Depois, quando o aluno (IA) for fazer uma prova, ele vai dar a resposta errada!

**Analogia do dia a dia:**
Imagine que você está treinando um cão-guia. Se alguém colocar sinais falsos no caminho durante o treinamento, o cão vai aprender a seguir os sinais errados. Quando ele estiver trabalhando de verdade, vai levar a pessoa cega para o lugar errado!

**Por que isso é perigoso?**

Uma IA com dados envenenados pode:
- Dar diagnósticos médicos errados
- Aprovar empréstimos para pessoas que não deveriam
- Classificar emails como spam quando são legítimos
- Tomar decisões baseadas em informações falsas

---

**🖼️ Sugestão de imagem**: Um hacker "envenenando" dados de treinamento com informações falsas

## **🔧 Setup Inicial - Preparando o Terreno**

Primeiro, vamos configurar nosso ambiente para estudar Data Poisoning:

In [ ]:
# 🔧 SETUP INICIAL - DATA POISONING
import os
import re
import json
import logging
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# Configurando logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('data_poisoning_log.txt'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Configurando estilo dos gráficos
plt.style.use('default')
sns.set_palette("husl")

print("☠️ Ambiente de Data Poisoning configurado!")
print("📝 Logs serão salvos em 'data_poisoning_log.txt'")

## **�� Configurando o LLM**

Vamos configurar nosso LLM para testar técnicas de Data Poisoning:

In [ ]:
# �� CONFIGURANDO O LLM
from dotenv import load_dotenv
load_dotenv()

def get_llm_colab():
    """Retorna o melhor LLM disponível no Colab"""
    
    # Tentativa 1: OpenAI
    try:
        from langchain_openai import ChatOpenAI
        api_key = os.getenv("OPENAI_API_KEY")
        if api_key:
            print("�� Usando OpenAI GPT-3.5-turbo")
            return ChatOpenAI(
                model="gpt-3.5-turbo",
                temperature=0.7,
                api_key=api_key
            )
    except Exception as e:
        print(f"⚠️  OpenAI não disponível: {e}")
    
    # Tentativa 2: Hugging Face
    try:
        from langchain_community.llms import HuggingFaceHub
        token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
        if token:
            print("🤗 Usando Hugging Face (google/flan-t5-base)")
            return HuggingFaceHub(
                repo_id="google/flan-t5-base",
                model_kwargs={"temperature": 0.7, "max_length": 512}
            )
    except Exception as e:
        print(f"⚠️  Hugging Face não disponível: {e}")
    
    # Fallback: Simulação
    print("🔄 Usando modo simulação (sem API)")
    return None

# Configurando o LLM
llm = get_llm_colab()

if llm:
    print("✅ LLM configurado com sucesso!")
else:
    print("✅ Modo simulação ativado - você pode continuar o curso normalmente!")

## **☠️ Exemplo Prático 1: Data Poisoning Simples**

Vamos criar um exemplo simples de Data Poisoning usando um classificador de emails. É como ensinar um filtro de spam a classificar emails legítimos como spam:

In [ ]:
# ☠️ EXEMPLO 1: DATA POISONING SIMPLES

class SistemaEmailSeguro:
    def __init__(self):
        # Dados de treinamento limpos (emails legítimos vs spam)
        self.dados_limpos = {
            'legitimo': [
                'Reunião amanhã às 10h',
                'Relatório mensal pronto',
                'Projeto finalizado com sucesso',
                'Convite para evento da empresa',
                'Atualização do sistema',
                'Feedback sobre sua apresentação',
                'Novo contrato assinado',
                'Análise de mercado concluída'
            ],
            'spam': [
                'Ganhe dinheiro rápido',
                'Viagra barato online',
                'Herança de 1 milhão de dólares',
                'Empréstimo sem comprovação',
                'Segredo para emagrecer',
                'Loteria garantida',
                'Produto milagroso',
                'Oferta por tempo limitado'
            ]
        }
        
        # Dados envenenados (serão adicionados depois)
        self.dados_envenenados = []
        
        # Modelo de classificação
        self.modelo = None
        self.acuracia_original = 0
        self.acuracia_envenenada = 0
    
    def criar_features(self, texto):
        """Cria features simples para classificação"""
        features = {
            'tem_dinheiro': 1 if 'dinheiro' in texto.lower() else 0,
            'tem_ganhe': 1 if 'ganhe' in texto.lower() else 0,
            'tem_reuniao': 1 if 'reunião' in texto.lower() else 0,
            'tem_projeto': 1 if 'projeto' in texto.lower() else 0,
            'tem_empresa': 1 if 'empresa' in texto.lower() else 0,
            'tem_viagra': 1 if 'viagra' in texto.lower() else 0,
            'tem_heranca': 1 if 'herança' in texto.lower() else 0,
            'tem_emprestimo': 1 if 'empréstimo' in texto.lower() else 0,
            'comprimento': len(texto),
            'maiusculas': sum(1 for c in texto if c.isupper())
        }
        return list(features.values())
    
    def preparar_dados(self, dados_limpos=True):
        """Prepara dados para treinamento"""
        X = []
        y = []
        
        # Adicionando dados limpos
        for email in self.dados_limpos['legitimo']:
            X.append(self.criar_features(email))
            y.append(0)  # 0 = legítimo
        
        for email in self.dados_limpos['spam']:
            X.append(self.criar_features(email))
            y.append(1)  # 1 = spam
        
        # Adicionando dados envenenados se solicitado
        if not dados_limpos:
            for email, label in self.dados_envenenados:
                X.append(self.criar_features(email))
                y.append(label)
        
        return np.array(X), np.array(y)
    
    def treinar_modelo(self, dados_limpos=True):
        """Treina o modelo de classificação"""
        X, y = self.preparar_dados(dados_limpos)
        
        # Dividindo em treino e teste
        X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Treinando modelo
        self.modelo = RandomForestClassifier(n_estimators=100, random_state=42)
        self.modelo.fit(X_treino, y_treino)
        
        # Avaliando
        y_pred = self.modelo.predict(X_teste)
        acuracia = accuracy_score(y_teste, y_pred)
        
        if dados_limpos:
            self.acuracia_original = acuracia
            print(f"✅ Modelo treinado com dados limpos - Acurácia: {acuracia:.2f}")
        else:
            self.acuracia_envenenada = acuracia
            print(f"☠️ Modelo treinado com dados envenenados - Acurácia: {acuracia:.2f}")
        
        return acuracia
    
    def classificar_email(self, email):
        """Classifica um email como legítimo ou spam"""
        if self.modelo is None:
            return "Modelo não treinado"
        
        features = self.criar_features(email)
        predicao = self.modelo.predict([features])[0]
        
        return "SPAM" if predicao == 1 else "LEGÍTIMO"

# Testando o sistema
print("📧 Testando sistema de classificação de emails...")
sistema = SistemaEmailSeguro()

# Treinando com dados limpos
print("\n🟢 Treinando com dados limpos...")
sistema.treinar_modelo(dados_limpos=True)

# Testando classificação
emails_teste = [
    "Reunião importante amanhã",
    "Ganhe dinheiro rápido e fácil",
    "Projeto finalizado com sucesso",
    "Viagra barato online"
]

print("\n📧 Testando classificação com dados limpos:")
for email in emails_teste:
    resultado = sistema.classificar_email(email)
    print(f"   '{email}' -> {resultado}")

print(f"\n�� Acurácia original: {sistema.acuracia_original:.2f}")

### **✅ Como o sistema deveria funcionar:**

O sistema de classificação de emails está funcionando corretamente! Ele consegue distinguir entre emails legítimos (como "Reunião importante amanhã") e spam (como "Ganhe dinheiro rápido").

**Agora vamos envenenar os dados!** ☠️

In [ ]:
# ☠️ APLICANDO DATA POISONING

def aplicar_data_poisoning(sistema):
    """Aplica Data Poisoning no sistema"""
    
    print("☠️ APLICANDO DATA POISONING...")
    print("=" * 50)
    
    # Estratégia 1: Adicionar emails legítimos como spam
    emails_legitimos_como_spam = [
        "Reunião importante amanhã às 10h",
        "Relatório mensal pronto para análise",
        "Projeto finalizado com sucesso",
        "Convite para evento da empresa",
        "Atualização do sistema de segurança",
        "Feedback sobre sua apresentação excelente",
        "Novo contrato assinado com cliente importante",
        "Análise de mercado concluída com sucesso"
    ]
    
    # Estratégia 2: Adicionar spam como legítimo
    spam_como_legitimo = [
        "Ganhe dinheiro rápido e fácil",
        "Viagra barato online - entrega rápida",
        "Herança de 1 milhão de dólares esperando",
        "Empréstimo sem comprovação de renda",
        "Segredo para emagrecer 10kg em 1 semana",
        "Loteria garantida - números da sorte",
        "Produto milagroso para cabelos",
        "Oferta por tempo limitado - não perca!"
    ]
    
    # Adicionando dados envenenados
    for email in emails_legitimos_como_spam:
        sistema.dados_envenenados.append((email, 1))  # Marcando como spam
        print(f"☠️ Adicionado como SPAM: '{email[:30]}...'")
    
    for email in spam_como_legitimo:
        sistema.dados_envenenados.append((email, 0))  # Marcando como legítimo
        print(f"☠️ Adicionado como LEGÍTIMO: '{email[:30]}...'")
    
    print(f"\n☠️ Total de dados envenenados adicionados: {len(sistema.dados_envenenados)}")
    
    # Treinando modelo com dados envenenados
    print("\n�� Retreinando modelo com dados envenenados...")
    acuracia_envenenada = sistema.treinar_modelo(dados_limpos=False)
    
    return acuracia_envenenada

# Aplicando Data Poisoning
acuracia_envenenada = aplicar_data_poisoning(sistema)

print(f"\n📊 COMPARAÇÃO:")
print(f"   �� Acurácia original: {sistema.acuracia_original:.2f}")
print(f"   ☠️ Acurácia envenenada: {acuracia_envenenada:.2f}")
print(f"   📉 Diferença: {sistema.acuracia_original - acuracia_envenenada:.2f}")

# Testando classificação com dados envenenados
print("\n📧 Testando classificação com dados envenenados:")
for email in emails_teste:
    resultado = sistema.classificar_email(email)
    print(f"   '{email}' -> {resultado}")

# Logging do ataque
logger.warning(f"Data Poisoning aplicado! Acurácia caiu de {sistema.acuracia_original:.2f} para {acuracia_envenenada:.2f}")

### **😱 O que acabou de acontecer?**

Acabamos de **envenenar** o sistema de classificação de emails! Agora ele está classificando emails legítimos como spam e spam como legítimo. É como se tivéssemos ensinado um cão-guia a levar pessoas para o lugar errado!

**Impactos do Data Poisoning:**
1. **Classificação incorreta** - O modelo agora faz erros que não fazia antes
2. **Acurácia reduzida** - A performance geral do sistema piorou
3. **Comportamento inesperado** - O modelo aprendeu padrões falsos
4. **Dificuldade de detecção** - O envenenamento pode ser sutil

---

**💡 Dica do Professor**: Data Poisoning é como colocar sal em vez de açúcar no café - pequenas mudanças podem ter grandes impactos!

## **�� Exemplo Prático 2: Modelo Inversion**

Agora vamos ver como hackers podem extrair informações sensíveis de um modelo treinado. É como tentar descobrir a receita de um bolo só provando o resultado final:

In [ ]:
# 🔍 EXEMPLO 2: MODELO INVERSION

class ModeloInversion:
    def __init__(self):
        # Dados sensíveis que foram usados no treinamento
        self.dados_sensiveis = {
            'usuarios': [
                {'nome': 'João Silva', 'idade': 35, 'salario': 5000, 'doenca': 'Diabetes'},
                {'nome': 'Maria Santos', 'idade': 28, 'salario': 4500, 'doenca': 'Hipertensão'},
                {'nome': 'Pedro Costa', 'idade': 42, 'salario': 7000, 'doenca': 'Asma'},
                {'nome': 'Ana Oliveira', 'idade': 31, 'salario': 5500, 'doenca': 'Depressão'},
                {'nome': 'Carlos Lima', 'idade': 39, 'salario': 6000, 'doenca': 'Diabetes'}
            ],
            'projetos': [
                {'nome': 'Projeto Alpha', 'orcamento': 100000, 'status': 'Confidencial'},
                {'nome': 'Projeto Beta', 'orcamento': 75000, 'status': 'Secreto'},
                {'nome': 'Projeto Gamma', 'orcamento': 150000, 'status': 'Classificado'}
            ]
        }
        
        # Modelo treinado (simulado)
        self.modelo_treinado = self.criar_modelo_simulado()
        
        # Contador de tentativas de inversão
        self.tentativas_inversao = 0
    
    def criar_modelo_simulado(self):
        """Cria um modelo simulado que 'lembra' dos dados de treinamento"""
        # Em um cenário real, isso seria um modelo de ML treinado
        return {
            'usuarios': self.dados_sensiveis['usuarios'],
            'projetos': self.dados_sensiveis['projetos'],
            'padroes_detectados': {
                'idade_media': 35,
                'salario_medio': 5600,
                'doencas_comuns': ['Diabetes', 'Hipertensão'],
                'orcamento_total': 325000
            }
        }
    
    def ataque_inversao_basico(self, consulta):
        """Ataque básico de inversão de modelo"""
        
        self.tentativas_inversao += 1
        
        # Simulando extração de informações
        if 'idade' in consulta.lower():
            return f"Idade média dos usuários: {self.modelo_treinado['padroes_detectados']['idade_media']} anos"
        
        if 'salario' in consulta.lower() or 'remuneracao' in consulta.lower():
            return f"Salário médio: R$ {self.modelo_treinado['padroes_detectados']['salario_medio']:.2f}"
        
        if 'doenca' in consulta.lower() or 'saude' in consulta.lower():
            return f"Doenças mais comuns: {', '.join(self.modelo_treinado['padroes_detectados']['doencas_comuns'])}"
        
        if 'projeto' in consulta.lower() or 'orcamento' in consulta.lower():
            return f"Orçamento total dos projetos: R$ {self.modelo_treinado['padroes_detectados']['orcamento_total']:,}"
        
        return "Informação não disponível"
    
    def ataque_inversao_avancado(self, consulta_especifica):
        """Ataque avançado de inversão - tentando extrair dados específicos"""
        
        self.tentativas_inversao += 1
        logger.warning(f"Tentativa de inversão avançada detectada: {consulta_especifica}")
        
        # Simulando extração de dados específicos
        consulta_lower = consulta_especifica.lower()
        
        # Tentando extrair dados de usuários específicos
        for usuario in self.dados_sensiveis['usuarios']:
            if usuario['nome'].lower() in consulta_lower:
                return f"Dados de {usuario['nome']}: Idade {usuario['idade']}, Salário R$ {usuario['salario']}, Doença: {usuario['doenca']}"
        
        # Tentando extrair dados de projetos específicos
        for projeto in self.dados_sensiveis['projetos']:
            if projeto['nome'].lower() in consulta_lower:
                return f"Projeto {projeto['nome']}: Orçamento R$ {projeto['orcamento']}, Status: {projeto['status']}"
        
        return "Dados específicos não encontrados"
    
    def detectar_tentativas_inversao(self):
        """Detecta tentativas de inversão de modelo"""
        
        palavras_suspeitas = [
            'idade', 'salario', 'remuneracao', 'doenca', 'saude',
            'projeto', 'orcamento', 'confidencial', 'secreto',
            'dados', 'informacao', 'extrair', 'revelar'
        ]
        
        return palavras_suspeitas

# Testando Modelo Inversion
print("🔍 Testando ataques de Modelo Inversion...")
inversor = ModeloInversion()

# Teste 1: Ataque básico
print("\n🔍 Teste 1: Ataque básico de inversão")
consultas_basicas = [
    "Qual é a idade média dos usuários?",
    "Qual é o salário médio?",
    "Quais são as doenças mais comuns?",
    "Qual é o orçamento total dos projetos?"
]

for consulta in consultas_basicas:
    resultado = inversor.ataque_inversao_basico(consulta)
    print(f"   🔍 '{consulta}' -> {resultado}")

# Teste 2: Ataque avançado
print("\n🚨 Teste 2: Ataque avançado de inversão")
consultas_avancadas = [
    "Quais são os dados de João Silva?",
    "Informações sobre o Projeto Alpha",
    "Dados de Maria Santos",
    "Detalhes do Projeto Beta"
]

for consulta in consultas_avancadas:
    resultado = inversor.ataque_inversao_avancado(consulta)
    print(f"   🚨 '{consulta}' -> {resultado}")

print(f"\n�� Estatísticas de Inversão:")
print(f"   🔍 Tentativas de inversão: {inversor.tentativas_inversao}")
print(f"   ⚠️  Dados sensíveis expostos: {len(consultas_basicas) + len(consultas_avancadas)}")

# Logging dos ataques
logger.warning(f"Ataques de Modelo Inversion detectados: {inversor.tentativas_inversao} tentativas")

### **😱 O que acabamos de ver?**

Acabamos de demonstrar como hackers podem **extrair informações sensíveis** de um modelo treinado! É como conseguir a receita de um bolo só provando o resultado final.

**Informações extraídas:**
- Idade média dos usuários
- Salários médios
- Doenças dos funcionários
- Orçamentos de projetos confidenciais
- Dados específicos de pessoas

**Por que isso é perigoso?**
1. **Violação de privacidade** - Dados pessoais expostos
2. **Vazamento de segredos** - Informações confidenciais da empresa
3. **Conformidade legal** - Violação de LGPD/GDPR
4. **Vantagem competitiva** - Concorrentes podem obter informações

---

**💡 Dica do Professor**: Modelo Inversion é como ser um "detetive de dados" - você usa o comportamento do modelo para descobrir o que ele aprendeu!

## **��️ Exemplo Prático 3: Proteção contra Data Poisoning**

Agora vamos criar um sistema que é resistente a Data Poisoning. É como ter um sistema imunológico para sua IA:

In [ ]:
# 🛡️ EXEMPLO 3: PROTEÇÃO CONTRA DATA POISONING

class SistemaProtegidoDataPoisoning:
    def __init__(self):
        self.dados_originais = []
        self.dados_validados = []
        self.modelo_protegido = None
        self.contador_suspeitos = 0
        self.limite_suspeitos = 5
    
    def validar_dados(self, dados):
        """Valida dados antes de usar no treinamento"""
        
        dados_validados = []
        dados_rejeitados = []
        
        for item in dados:
            # Verificação 1: Padrões suspeitos
            if self.detectar_padrao_suspeito(item):
                dados_rejeitados.append((item, 'Padrão suspeito'))
                continue
            
            # Verificação 2: Anomalias estatísticas
            if self.detectar_anomalia_estatistica(item):
                dados_rejeitados.append((item, 'Anomalia estatística'))
                continue
            
            # Verificação 3: Consistência com dados existentes
            if not self.verificar_consistencia(item):
                dados_rejeitados.append((item, 'Inconsistente'))
                continue
            
            # Se passou por todas as verificações
            dados_validados.append(item)
        
        return dados_validados, dados_rejeitados
    
    def detectar_padrao_suspeito(self, item):
        """Detecta padrões suspeitos nos dados"""
        
        # Padrões suspeitos para emails
        if isinstance(item, str):
            padroes_suspeitos = [
                r'ganhe\s+dinheiro\s+rapido',
                r'viagra\s+barato',
                r'heranca\s+de\s+\d+\s+milhao',
                r'emprestimo\s+sem\s+comprovacao',
                r'produto\s+milagroso',
                r'oferta\s+por\s+tempo\s+limitado'
            ]
            
            for padrao in padroes_suspeitos:
                if re.search(padrao, item.lower()):
                    return True
        
        return False
    
    def detectar_anomalia_estatistica(self, item):
        """Detecta anomalias estatísticas nos dados"""
        
        # Simulando detecção de anomalias
        if isinstance(item, str):
            # Verificar se o texto é muito diferente dos padrões normais
            palavras_normais = ['reunião', 'projeto', 'empresa', 'relatório', 'análise']
            palavras_suspeitas = ['dinheiro', 'ganhe', 'viagra', 'herança', 'milagroso']
            
            contador_normais = sum(1 for palavra in palavras_normais if palavra in item.lower())
            contador_suspeitas = sum(1 for palavra in palavras_suspeitas if palavra in item.lower())
            
            # Se tem mais palavras suspeitas que normais
            if contador_suspeitas > contador_normais:
                return True
        
        return False
    
    def verificar_consistencia(self, item):
        """Verifica consistência com dados existentes"""
        
        # Simulando verificação de consistência
        if len(self.dados_originais) > 0:
            # Verificar se o novo item é muito diferente dos existentes
            comprimento_medio = np.mean([len(str(d)) for d in self.dados_originais])
            comprimento_item = len(str(item))
            
            # Se o comprimento é muito diferente (anomalia)
            if abs(comprimento_item - comprimento_medio) > comprimento_medio * 2:
                return False
        
        return True
    
    def adicionar_dados_seguros(self, novos_dados):
        """Adiciona dados de forma segura"""
        
        print(f"🛡️ Validando {len(novos_dados)} novos itens...")
        
        dados_validados, dados_rejeitados = self.validar_dados(novos_dados)
        
        # Adicionando dados válidos
        self.dados_originais.extend(dados_validados)
        self.dados_validados.extend(dados_validados)
        
        # Logging de dados rejeitados
        if dados_rejeitados:
            self.contador_suspeitos += len(dados_rejeitados)
            logger.warning(f"Dados suspeitos rejeitados: {len(dados_rejeitados)} itens")
            
            for item, motivo in dados_rejeitados:
                logger.warning(f"Item rejeitado: '{str(item)[:50]}...' - Motivo: {motivo}")
        
        print(f"✅ {len(dados_validados)} itens adicionados com sucesso")
        print(f"🚫 {len(dados_rejeitados)} itens rejeitados como suspeitos")
        
        # Verificar se há muitas tentativas suspeitas
        if self.contador_suspeitos > self.limite_suspeitos:
            logger.critical(f"Muitas tentativas suspeitas detectadas! Possível ataque de Data Poisoning.")
            print(f"🚨 ALERTA: Possível ataque de Data Poisoning detectado!")
        
        return dados_validados, dados_rejeitados

# Testando o sistema protegido
print("🛡️ Testando sistema protegido contra Data Poisoning...")
sistema_protegido = SistemaProtegidoDataPoisoning()

# Dados normais
dados_normais = [
    "Reunião de equipe amanhã",
    "Relatório mensal pronto",
    "Projeto finalizado com sucesso",
    "Atualização do sistema"
]

# Dados suspeitos (tentativa de poisoning)
dados_suspeitos = [
    "Ganhe dinheiro rápido e fácil",
    "Viagra barato online",
    "Herança de 1 milhão de dólares",
    "Produto milagroso para emagrecer",
    "Oferta por tempo limitado"
]

# Teste 1: Adicionando dados normais
print("\n🟢 Teste 1: Adicionando dados normais")
validados, rejeitados = sistema_protegido.adicionar_dados_seguros(dados_normais)

# Teste 2: Tentativa de poisoning
print("\n☠️ Teste 2: Tentativa de Data Poisoning")
validados, rejeitados = sistema_protegido.adicionar_dados_seguros(dados_suspeitos)

print(f"\n�� RESULTADO DA PROTEÇÃO:")
print(f"   ✅ Dados válidos adicionados: {len(validados)}")
print(f"   �� Dados suspeitos rejeitados: {len(rejeitados)}")
print(f"   ��️ Sistema protegido funcionando!")

## **📊 Visualizando Ataques de Data Poisoning**

Vamos criar visualizações para entender melhor os padrões de Data Poisoning:

In [ ]:
# �� VISUALIZANDO ATAQUES DE DATA POISONING

# Dados simulados de ataques
tipos_poisoning = [
    "Label Flipping",
    "Feature Poisoning",
    "Backdoor Injection",
    "Clean Label Attack",
    "Model Inversion"
]

dificuldade_implementacao = [3, 6, 8, 7, 9]  # Escala 1-10
taxa_sucesso = [85, 60, 40, 70, 30]  # Percentual
dificuldade_deteccao = [4, 7, 5, 9, 6]  # Escala 1-10
impacto_dano = [7, 8, 9, 6, 8]  # Escala 1-10

# Criando gráficos
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Gráfico 1: Dificuldade de implementação
cores = ['#ff6b6b', '#4ecdc4', '#45b7d1', '#96ceb4', '#feca57']
bars1 = ax1.bar(tipos_poisoning, dificuldade_implementacao, color=cores, alpha=0.8)
ax1.set_title('Dificuldade de Implementação (1-10)', fontsize=14, fontweight='bold')
ax1.set_ylabel('Dificuldade')
ax1.set_ylim(0, 10)
ax1.tick_params(axis='x', rotation=45)

for bar, valor in zip(bars1, dificuldade_implementacao):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             f'{valor}', ha='center', va='bottom', fontweight='bold')

# Gráfico 2: Taxa de sucesso
bars2 = ax2.bar(tipos_poisoning, taxa_sucesso, color=cores, alpha=0.8)
ax2.set_title('Taxa de Sucesso dos Ataques (%)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Taxa de Sucesso (%)')
ax2.tick_params(axis='x', rotation=45)

for bar, valor in zip(bars2, taxa_sucesso):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{valor}%', ha='center', va='bottom', fontweight='bold')

# Gráfico 3: Dificuldade de detecção
bars3 = ax3.bar(tipos_poisoning, dificuldade_deteccao, color=cores, alpha=0.8)
ax3.set_title('Dificuldade de Detecção (1-10)', fontsize=14, fontweight='bold')
ax3.set_ylabel('Dificuldade de Detecção')
ax3.set_ylim(0, 10)
ax3.tick_params(axis='x', rotation=45)

for bar, valor in zip(bars3, dificuldade_deteccao):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             f'{valor}', ha='center', va='bottom', fontweight='bold')

# Gráfico 4: Impacto do dano
bars4 = ax4.bar(tipos_poisoning, impacto_dano, color=cores, alpha=0.8)
ax4.set_title('Impacto do Dano (1-10)', fontsize=14, fontweight='bold')
ax4.set_ylabel('Impacto')
ax4.set_ylim(0, 10)
ax4.tick_params(axis='x', rotation=45)

for bar, valor in zip(bars4, impacto_dano):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             f'{valor}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("📊 Análise dos Ataques de Data Poisoning:")
print("=" * 60)
for i, tipo in enumerate(tipos_poisoning):
    print(f"\n{i+1}. {tipo}:")
    print(f"   🔧 Dificuldade de Implementação: {dificuldade_implementacao[i]}/10")
    print(f"   �� Taxa de Sucesso: {taxa_sucesso[i]}%")
    print(f"   �� Dificuldade de Detecção: {dificuldade_deteccao[i]}/10")
    print(f"   �� Impacto do Dano: {impacto_dano[i]}/10")
    
    # Recomendação baseada nos dados
    risco_total = (dificuldade_implementacao[i] + dificuldade_deteccao[i] + impacto_dano[i]) / 3
    if risco_total > 7:
        print(f"   �� ALERTA: Alto risco - requer proteção especial!")
    elif risco_total > 5:
        print(f"   ⚠️  Risco médio - monitoramento necessário")
    else:
        print(f"   ✅ Risco baixo - proteção padrão suficiente")

## **🔍 Teste Rápido - Identificando Data Poisoning**

Agora é sua vez! Vamos testar se você consegue identificar tentativas de Data Poisoning:

In [ ]:
# 🔍 TESTE RÁPIDO - IDENTIFICANDO DATA POISONING

def testar_reconhecimento_poisoning():
    """Testa a capacidade de reconhecer Data Poisoning"""
    
    dados_teste = [
        # Dados normais
        "Reunião de equipe amanhã às 10h",
        "Relatório mensal pronto para análise",
        "Projeto finalizado com sucesso",
        "Atualização do sistema de segurança",
        
        # Tentativas de poisoning
        "Ganhe dinheiro rápido e fácil",
        "Viagra barato online - entrega rápida",
        "Herança de 1 milhão de dólares esperando",
        "Empréstimo sem comprovação de renda",
        "Produto milagroso para emagrecer 10kg",
        "Oferta por tempo limitado - não perca!",
        
        # Dados ambíguos
        "Análise de mercado com resultados positivos",
        "Novo produto lançado com sucesso",
        "Relatório financeiro com lucros recorde"
    ]
    
    classificacoes = [
        "Normal", "Normal", "Normal", "Normal",
        "Poisoning", "Poisoning", "Poisoning", "Poisoning", "Poisoning", "Poisoning",
        "Ambíguo", "Ambíguo", "Ambíguo"
    ]
    
    print("🔍 TESTE DE RECONHECIMENTO DE DATA POISONING")
    print("=" * 60)
    print("\nAnalise cada dado e classifique como:")
    print("   🟢 Normal - Dado legítimo")
    print("   ☠️ Poisoning - Tentativa de envenenamento")
    print("   ⚠️  Ambíguo - Pode ser legítimo ou suspeito")
    
    acertos = 0
    total = len(dados_teste)
    
    for i, (dado, classificacao_correta) in enumerate(zip(dados_teste, classificacoes), 1):
        print(f"\n{i}. Dado: \"{dado}\"")
        
        # Simulando análise automática
        palavras_suspeitas = [
            "ganhe dinheiro", "viagra barato", "herança", "empréstimo sem",
            "produto milagroso", "oferta por tempo limitado"
        ]
        
        dado_lower = dado.lower()
        suspeitas = [palavra for palavra in palavras_suspeitas if palavra in dado_lower]
        
        if suspeitas:
            classificacao_auto = "Poisoning"
            print(f"   ☠️ Classificação Automática: Poisoning (palavras suspeitas: {suspeitas})")
        elif "lucro" in dado_lower or "record" in dado_lower:
            classificacao_auto = "Ambíguo"
            print(f"   ⚠️  Classificação Automática: Ambíguo")
        else:
            classificacao_auto = "Normal"
            print(f"   🟢 Classificação Automática: Normal")
        
        # Verificando acerto
        if classificacao_auto == classificacao_correta:
            acertos += 1
            print(f"   ✅ CORRETO!")
        else:
            print(f"   ❌ INCORRETO! Classificação correta: {classificacao_correta}")
    
    # Resultado final
    print(f"\n RESULTADO FINAL:")
    print(f"   Acertos: {acertos}/{total}")
    print(f"   Taxa de acerto: {(acertos/total)*100:.1f}%")
    
    if acertos >= total * 0.8:
        print("   🏆 EXCELENTE! Você tem um bom olho para detectar Data Poisoning!")
    elif acertos >= total * 0.6:
        print("   👍 BOM! Você está no caminho certo!")
    else:
        print("   Continue estudando! A prática leva à perfeição!")
    
    return acertos, total

# Executando o teste
acertos, total = testar_reconhecimento_poisoning()

# Logging do resultado
logger.info(f"Teste de reconhecimento de Data Poisoning concluído: {acertos}/{total} acertos")

## **🏆 Desafio do Módulo - Sistema Anti-Data Poisoning**

Agora é hora de colocar em prática! Vamos criar um sistema completo de proteção contra Data Poisoning:

In [ ]:
# 🏆 DESAFIO DO MÓDULO - SISTEMA ANTI-DATA POISONING

class SistemaAntiDataPoisoning:
    def __init__(self):
        # TODO: Implemente as variáveis necessárias
        # - Sistema de validação de dados
        # - Detecção de anomalias
        # - Sistema de scoring de confiança
        # - Logging de tentativas suspeitas
        pass
    
    def validar_dados_avancado(self, dados):
        """Valida dados usando múltiplas técnicas"""
        # TODO: Implemente validação avançada
        # - Análise estatística
        # - Detecção de outliers
        # - Verificação de consistência
        # - Análise de padrões suspeitos
        pass
    
    def calcular_score_confianca(self, dados):
        """Calcula um score de confiança para os dados"""
        # TODO: Implemente cálculo de confiança
        # - Qualidade dos dados
        # - Consistência com histórico
        # - Ausência de padrões suspeitos
        # - Fonte dos dados
        pass
    
    def detectar_anomalias_estatisticas(self, dados):
        """Detecta anomalias estatísticas nos dados"""
        # TODO: Implemente detecção de anomalias
        # - Análise de distribuição
        # - Detecção de outliers
        # - Mudanças bruscas de padrão
        # - Inconsistências temporais
        pass
    
    def processar_dados_seguros(self, novos_dados):
        """Processa dados com todas as proteções"""
        # TODO: Implemente processamento seguro
        # - Validação de dados
        # - Detecção de anomalias
        # - Cálculo de confiança
        # - Logging de suspeitas
        # - Ação apropriada baseada no risco
        pass

print(" DESAFIO: Implemente o sistema anti-Data Poisoning acima!")
print("💡 Dicas:")
print("   - Use estatísticas para detectar outliers")
print("   - Implemente múltiplas camadas de validação")
print("   - Use machine learning para detectar padrões suspeitos")
print("   - Implemente sistema de scoring de confiança")
print("   - Log todas as tentativas suspeitas")
print("   - Implemente diferentes níveis de ação baseados no risco")

## **📈 Métricas de Proteção contra Data Poisoning**

Vamos criar um dashboard para monitorar a proteção contra Data Poisoning:

In [ ]:
# 📈 MÉTRICAS DE PROTEÇÃO CONTRA DATA POISONING

# Dados simulados de métricas
dados_metricas = {
    'dados_recebidos': 1000,
    'dados_validados': 950,
    'dados_rejeitados': 50,
    'tentativas_poisoning': 25,
    'taxa_deteccao': 96.0,
    'falsos_positivos': 15,
    'tempo_medio_validacao': 0.3
}

# Criando dashboard
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Gráfico 1: Distribuição de dados
labels = ['Validados', 'Rejeitados', 'Poisoning Detectado']
sizes = [dados_metricas['dados_validados'], 
         dados_metricas['dados_rejeitados'] - dados_metricas['tentativas_poisoning'],
         dados_metricas['tentativas_poisoning']]
colors = ['#4ecdc4', '#feca57', '#ff6b6b']

ax1.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax1.set_title('Distribuição de Dados Processados', fontsize=14, fontweight='bold')

# Gráfico 2: Taxa de detecção ao longo do tempo
dias = list(range(1, 31))
taxa_deteccao_diaria = [94 + np.random.normal(0, 1.5) for _ in dias]
taxa_deteccao_diaria = [max(90, min(100, taxa)) for taxa in taxa_deteccao_diaria]

ax2.plot(dias, taxa_deteccao_diaria, 'o-', color='#4ecdc4', linewidth=2, markersize=6)
ax2.set_title('Taxa de Detecção Diária (%)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Dia')
ax2.set_ylabel('Taxa de Detecção (%)')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(85, 100)

# Gráfico 3: Tipos de ataques detectados
tipos_ataque = ['Label Flipping', 'Feature Poisoning', 'Backdoor', 'Clean Label', 'Model Inversion']
contagem_ataques = [8, 6, 4, 5, 2]

bars = ax3.bar(tipos_ataque, contagem_ataques, color=['#ff6b6b', '#feca57', '#45b7d1', '#96ceb4', '#4ecdc4'], alpha=0.8)
ax3.set_title('Tipos de Ataques Detectados', fontsize=14, fontweight='bold')
ax3.set_ylabel('Número de Tentativas')
ax3.tick_params(axis='x', rotation=45)

for bar, valor in zip(bars, contagem_ataques):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2, 
             str(valor), ha='center', va='bottom', fontweight='bold')

# Gráfico 4: Métricas principais
ax4.axis('off')
metricas_texto = f"""
🛡️ DASHBOARD ANTI-DATA POISONING
{'='*40}

�� Dados Recebidos: {dados_metricas['dados_recebidos']}
✅ Dados Validados: {dados_metricas['dados_validados']}
🚫 Dados Rejeitados: {dados_metricas['dados_rejeitados']}
☠️ Poisoning Detectado: {dados_metricas['tentativas_poisoning']}

📈 Taxa de Detecção: {dados_metricas['taxa_deteccao']}%
⚠️ Falsos Positivos: {dados_metricas['falsos_positivos']}
⏱️ Tempo Médio Validação: {dados_metricas['tempo_medio_validacao']}s

✅ Status: SISTEMA PROTEGIDO
🛡️ Última Atualização: {datetime.now().strftime('%d/%m/%Y %H:%M')}
"""

ax4.text(0.1, 0.9, metricas_texto, transform=ax4.transAxes, fontsize=12, 
         verticalalignment='top', fontfamily='monospace', fontweight='bold')

plt.tight_layout()
plt.show()

print("📊 Dashboard Anti-Data Poisoning gerado com sucesso!")
print(f"🎯 Taxa de detecção atual: {dados_metricas['taxa_deteccao']}%")
print(f"��️ Sistema protegido contra envenenamento de dados!")

---

## ** Resumo do Módulo**

Neste módulo, aprendemos:

### **☠️ Conceitos de Data Poisoning:**
- **O que é Data Poisoning** - Envenenamento de dados de treinamento
- **Tipos de ataques** - Label flipping, feature poisoning, backdoor injection
- **Modelo Inversion** - Extração de informações sensíveis de modelos treinados

### **🛠️ Ferramentas Aprendidas:**
- Validação de dados antes do treinamento
- Detecção de anomalias estatísticas
- Sistema de scoring de confiança
- Proteção contra extração de informações
- Dashboard de métricas de proteção

### **🔬 Técnicas Práticas:**
- Como identificar dados envenenados
- Como proteger sistemas contra poisoning
- Como detectar tentativas de inversão de modelo
- Como monitorar ataques em tempo real

### ** Próximos Passos:**
- No próximo módulo, vamos aprender sobre **Observabilidade e Monitoramento**
- Como monitorar sistemas de IA em tempo real
- Técnicas de logging e alertas

---

**💡 Dica do Professor**: Data Poisoning é como ser um "envenenador de dados" - você precisa entender como os dados afetam o treinamento para protegê-los!

**🚀 Próximo módulo**: [Módulo 4: Observabilidade e Monitoramento](./04_observabilidade.ipynb)

---

**🎉 Parabéns! Você completou o Módulo 3!** ☠️

**🏆 Conquista Desbloqueada**: "Guardião dos Dados" - Você agora pode proteger sistemas de IA contra envenenamento de dados!